In [0]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers.core import Dense, Activation, RepeatVector
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

np.random.seed(0)
N = 200000
N_train = int(N*0.9)
N_validation = N-N_train

'''
データの生成
'''
def padding(chars, maxlen):
    return chars + " " * (maxlen-len(chars))

def n(digits=3):
    number = ""
    for i in range(np.random.randint(1, digits+1)):     # 最大桁数digitsの数字（文字列）を生成
        number += np.random.choice(list("0123456789"))
    return int(number)

digits = 3      # 最大桁数
input_digits = digits * 2 + 1       # 例: 123+456 -> 7桁
output_digits = digits + 1          # 500+500=1000以上で4桁

added = set()
questions = []
answers = []

while len(questions) < N:
    a, b = n(), n()
    pair = tuple(sorted((a, b)))
    if pair in added:
        continue

    question = "{}+{}".format(a, b)
    question = padding(question, input_digits)      # 足りない桁を穴埋め
    answer = str(a + b)
    answer = padding(answer, output_digits)         # 足りない桁を穴埋め

    added.add(pair)
    questions.append(question)
    answers.append(answer)

chars = "0123456789+ "
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

X = np.zeros((len(questions), input_digits, len(chars)), dtype=np.integer)
Y = np.zeros((len(answers), output_digits, len(chars)), dtype=np.integer)

for i in range(N):
    for t, char in enumerate(questions[i]):
        X[i, t, char_indices[char]] = 1         # one-hot表現
    for t, char in enumerate(answers[i]):
        Y[i, t, char_indices[char]]  =1

# X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, train_size=N_train)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=N_train)

X_train, X_validation, Y_train, Y_validation = train_test_split(X_train, Y_train, test_size=N_validation)



"""
モデル設定
"""
n_in = len(chars)
n_hidden = 128
n_out = len(chars)

early_stopping = EarlyStopping(monitor= "val_loss", patience= 10, verbose= 1)

model = Sequential()
# Encoder
model.add(LSTM(n_hidden, input_shape= (input_digits, n_in)))
# Decoder
model.add(RepeatVector(output_digits))
model.add(LSTM(n_hidden, return_sequences=True))

model.add(TimeDistributed(Dense(n_out)))
model.add(Activation("softmax"))

model.compile(  loss= "categorical_crossentropy",
                optimizer= Adam(lr= 0.001, beta_1= 0.9, beta_2= 0.999),
                metrics= ["accuracy"])

"""
モデル学習
"""
epochs = 200
batch_size = 3000
hist = model.fit(   X_train, Y_train,
                    batch_size= batch_size,
                    epochs= epochs,
                    validation_data= (X_validation, Y_validation),
                    callbacks= [early_stopping])

"""
学習の進み具合を可視化
"""
acc = hist.history["val_acc"]
loss = hist.history["val_loss"]
plt.rc('font', family='serif')
fig = plt.figure()
ax_acc = fig.add_subplot(111)
ax_acc.plot(range(len(acc)), acc, label='loss', color='black')
ax_loss = ax_acc.twinx()
ax_loss.plot(range(len(loss)), loss, label='acc', color='gray')
plt.xlabel('epochs')
plt.show()

'''
予測精度の評価
'''
loss_and_metrics = model.evaluate(X_test, Y_test)
print(loss_and_metrics)

# 検証データからランダムに問題を選んで答え合わせ
for i in range(10):
    index = np.random.randint(0, N_validation)
    question = X_validation[np.array([index])]
    answer = Y_validation[np.array([index])]
    prediction = model.predict_classes(question, verbose=0)

    question = question.argmax(axis=-1)
    answer = answer.argmax(axis=-1)

    q = ''.join(indices_char[i] for i in question[0])
    a = ''.join(indices_char[i] for i in answer[0])
    p = ''.join(indices_char[i] for i in prediction[0])

    print('-' * 10)
    print('Q:  ', q)
    print('A:  ', p)
    print('T/F:', end=' ')
    if a == p:
        print('T')
    else:
        print('F')
print('-' * 10)